# ディープラーニングフレームワーク1

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

### 【問題1】スクラッチを振り返る

ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

- 最適化手法（optimizer：SGD, AdaGrad）が必要
- 活性化関数（activation：Sigmoid, Tanh, ReLU, Softmax）が必要
- 目的関数（loss：cross_entropy）が必要
- フォワードプロパゲーション、バックプロパゲーションの実装が必要

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

### データセットの用意

以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。

目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。

- Iris-versicolor
- Iris-virginica

### 【問題2】スクラッチとTensorFlowの対応を考える

以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [2]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""


# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)


# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
print(y.shape)


# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1


# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # tf.addと+は等価である
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

(100, 1)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, loss : 26.2887, val_loss : 68.5501, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 0.0000, val_loss : 1.0745, acc : 1.000, val_acc : 0.938
Epoch 2, loss : 0.0000, val_loss : 3.4347, acc : 1.000, val_acc : 0.875
Epoch 3, loss : 0.0000, val_loss : 2.4012, acc : 1.000, val_acc : 0.812
Epoch 4, loss : 0.0000, val_loss : 0.9788, acc : 1.000, val_acc : 0.938
Epoch 5, loss : 0.0000, val_loss : 0.0250, acc : 1.000, val_acc : 1.000
Epoch 6, loss : 0.0000, val_loss : 0.2106, acc : 1.000, val_acc : 0.938
Epoch 7, loss : 0.0000, val_loss : 4.1188, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.0000, val_loss : 1.9584, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 0.8481, acc : 1.000, val_acc : 0.938
test_acc : 0.900


**重みの初期化、エポックのループ、最適化手法、活性化関数、目的関数などは個別に指定が必要であり、この点はスクラッチ同様である。ただし、フォワードプロパゲーション/バックプロパゲーションについては、ネットワーク構造に対応して自動的にバックプロパゲーションを行う仕組みとなっており、別々の実装は必要ではない。
一方で、Variable（変数）やplaceholder（プレースホルダー）の操作を宣言すること、計算の際にSession（セッション）オブジェクトを通して計算グラフを構築してrun（実行）する点などがスクラッチとは異なっている。（なお、TensorFlow2.xでは、placeholderやSessionは廃止されている。）**

### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。
2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

In [4]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)

# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)


# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]
#print(y.shape)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1


# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # tf.addと+は等価である
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : -86.9101, val_loss : -234.4330, acc : 0.833, val_acc : 0.667
Epoch 1, loss : -161.7518, val_loss : -427.0560, acc : 0.833, val_acc : 0.667
Epoch 2, loss : -253.5382, val_loss : -662.4175, acc : 0.833, val_acc : 0.667
Epoch 3, loss : -363.5638, val_loss : -950.2044, acc : 0.833, val_acc : 0.708
Epoch 4, loss : -509.4829, val_loss : -1353.7472, acc : 0.833, val_acc : 0.708
Epoch 5, loss : -708.8834, val_loss : -1885.6376, acc : 0.833, val_acc : 0.708
Epoch 6, loss : -973.4127, val_loss : -2597.2380, acc : 0.833, val_acc : 0.708
Epoch 7, loss : -1327.9281, val_loss : -3533.4211, acc : 0.833, val_acc : 0.708
Epoch 8, loss : -1783.6195, val_loss : -4768.4917, acc : 0.833, val_acc : 0.708
Epoch 9, loss : -2380.0469, val_loss : -6347.2051, acc : 0.833, val_acc : 0.708
test_acc : 0.733


### 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

In [5]:
# データセットの読み込み
df4 = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")

# データフレームから「GrLivArea」と「YearBuilt」を抜き出して配列に変換
X = df4.loc[:,["GrLivArea","YearBuilt"]].values
# データフレームから「SalePrice」を抜き出して、配列に変換
y = df4.loc[:,["SalePrice"]].values

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 説明変数を標準化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.losses.mean_squared_error(labels=Y, predictions=logits) # MSEを使用

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
# correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax((logits), axis=1))

# 指標値計算（目的関数でMSEを用いたため、RMSEで評価する）
accuracy = tf.sqrt(tf.losses.mean_squared_error(labels=Y, predictions=logits)) # RMSE

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    loss_list = []
    val_loss_list = []
    print("loss = MSE, score = RMSE")
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_score = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, score = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_score += score
        total_loss /= n_samples
        total_score /= n_samples
        loss_list.append(loss)
        val_loss, val_score = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        val_loss_list.append(val_loss)
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, score : {:.3f}, val_score : {:.3f}".format(epoch, loss, val_loss, score, val_score))
    test_score = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_score : {:.3f}".format(test_score))

loss = MSE, score = RMSE
Epoch 0, loss : 28568506368.0000, val_loss : 33523607552.0000, score : 169022.203, val_score : 183094.531
Epoch 1, loss : 9256515584.0000, val_loss : 11572856832.0000, score : 96210.789, val_score : 107577.211
Epoch 2, loss : 416273920.0000, val_loss : 1626998016.0000, score : 20402.791, val_score : 40336.062
Epoch 3, loss : 494852864.0000, val_loss : 1559241984.0000, score : 22245.289, val_score : 39487.238
Epoch 4, loss : 525332928.0000, val_loss : 1536231680.0000, score : 22920.143, val_score : 39194.793
Epoch 5, loss : 524876864.0000, val_loss : 1518929536.0000, score : 22910.191, val_score : 38973.445
Epoch 6, loss : 515634112.0000, val_loss : 1503702784.0000, score : 22707.578, val_score : 38777.605
Epoch 7, loss : 502369344.0000, val_loss : 1490677888.0000, score : 22413.598, val_score : 38609.297
Epoch 8, loss : 485292736.0000, val_loss : 1479856768.0000, score : 22029.361, val_score : 38468.906
Epoch 9, loss : 473071712.0000, val_loss : 1469643264.0000

### 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [6]:
# mnistのデータセットを読み込む
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# データセットの下準備を行う
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# trainとtestに分割する
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(X_train.shape) # (48000, 784)
print(X_val.shape) # (12000, 784)

Using TensorFlow backend.


(48000, 28, 28, 1)
(12000, 28, 28, 1)


In [7]:
# OneHotEncoderのサンプルコードを実行する
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64

# 検証データ、テストデータもone-hot表現に変換する
y_val_one_hot = enc.fit_transform(y_val[:, np.newaxis])
print(y_val_one_hot.shape)
y_test_one_hot = enc.fit_transform(y_test[:, np.newaxis])
print(y_test_one_hot.shape)

(48000,)
(48000, 10)
float64
(12000, 10)
(10000, 10)


In [8]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 200
num_epochs = 30
n_filter = 3 # 3×3フィルタ
n_hidden1 = 32
n_hidden2 = 64
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)

def example_net(X):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_filter, n_filter, 1, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([14*14*32, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    conv_1 = tf.nn.conv2d(X, weights['w1'], strides=[1, 1, 1, 1], padding="SAME") + biases['b1'] # 畳み込み層
    conv_1 = tf.nn.relu(conv_1)
    pool_1 = tf.nn.max_pool(conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME") # プーリング層
    fc_flat = tf.reshape(pool_1, [-1, 14*14*32]) # 平滑化層（14=28/2 ∵2×2プーリング）
    fc_1 = tf.matmul(fc_flat, weights['w2']) + biases['b2']
    fc_1 = tf.nn.relu(fc_1)
    fc_2 = tf.matmul(fc_1, weights['w3']) + biases['b3']
    return fc_2

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(Y, logits)) # ソフトマックス関数を使用

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax((logits), axis=1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    loss_list = []
    val_loss_list = []
    print("loss = cross_entropy")
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        loss_list.append(loss)
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_one_hot})
        val_loss_list.append(val_loss)
        if epoch % 3 == 0:
            print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

loss = cross_entropy
Epoch 0, loss : 1.8461, val_loss : 1.9166, acc : 0.290, val_acc : 0.297
Epoch 3, loss : 0.9918, val_loss : 1.1057, acc : 0.610, val_acc : 0.634
Epoch 6, loss : 0.5594, val_loss : 0.7236, acc : 0.755, val_acc : 0.777
Epoch 9, loss : 0.3002, val_loss : 0.4343, acc : 0.910, val_acc : 0.917
Epoch 12, loss : 0.2185, val_loss : 0.3676, acc : 0.935, val_acc : 0.932
Epoch 15, loss : 0.1303, val_loss : 0.2556, acc : 0.970, val_acc : 0.950
Epoch 18, loss : 0.0909, val_loss : 0.2324, acc : 0.975, val_acc : 0.956
Epoch 21, loss : 0.0708, val_loss : 0.2364, acc : 0.985, val_acc : 0.959
Epoch 24, loss : 0.1389, val_loss : 0.2418, acc : 0.965, val_acc : 0.955
Epoch 27, loss : 0.0341, val_loss : 0.2127, acc : 0.990, val_acc : 0.960
test_acc : 0.959
